### Imports & Settings

In [1]:
from collections import Counter
import time
%run nn-model.py

### Parameters

In [2]:
epochs_n = 1
learning_rate = 0.1
seed = 19101995

### Model classes

In [3]:
#Server 
class FairServer:
    def __init__(self, model):
        self.model = LearningModel(model)
        self.workers = []
        self.groups = Counter()
        
    def register_worker(self, worker):
        self.workers += [worker]
        
    def register_feature_group(self, group):
        self.groups.update(group)

    def send_model(self, worker):
        worker.load_model(self.model)
        
    def load_weights(self, W):
        self.model.set_weights(W)
        
    def request_training(self, w):
        w.train()
    
    def train(self):
        for w in self.workers:
            self.send_model(w)
            self.request_training(w)
            
    def fair_metrics(self):
        return 'Fair metrics: '
    
    def predict(self, x):
        return self.model.predict(x)

In [4]:
#Worker 
class Worker():
    def __init__(self, server, x, y, secret_feature):
        self.server = server
        self.model = {}
        self.x = x
        self.y = y
        self.secret_feature = secret_feature
        
    def send_registration(self):
        self.server.register_worker(self)
        
    def send_feature_group(self):
        self.server.register_feature_group(self.secret_feature)
        
    def load_model(self, model):
        self.model = model
        
    def send_weights(self, W):
        self.server.load_weights(W)
        
    def train(self):
        self.model.train(self.x, self.y)
        self.send_weights(self.model.get_weights())
        self.send_feature_group()

In [5]:
class LearningModel:
    def __init__(self, model, epochs=epochs_n, learning_rate=learning_rate):
        self.model = model
        self.epochs = epochs
        self.learning_rate = learning_rate
    
    def set_weights(self, W):
        self.model.load_weights(W)
        
    def get_weights(self):
        return self.model.get_weights()
    
    def train(self, X, y):
        self.model.fit(X, y, self.epochs, self.learning_rate)
    
    def predict(self, x):
        return self.model.predict(x)

### Utils functions

In [6]:
#Create a server
def create_server(model):
    return FairServer(model)

#Create workers
def create_workers(server, X, y, secret_feature):
    return[Worker(server, X[i], y[i], secret_feature[i]) for i in range(len(X))]

#Register all workers
def register_workers(workers):
    for worker in workers:
        worker.send_registration()
        
def start_training(server):
    start_train = time.time()
    server.train()
    end_train = time.time()
    #print('Training time:' + str(end_train - start_train))
    
def znp_fed_training(model, X, y, secret):
    s = create_server(model)
    ws = create_workers(s, X, y, secret)
    register_workers(ws)
    start_training(s)
    #print(s.fair_metrics())
    return s

### Model for reference

In [7]:
# network (not federated)
net = Network(seed=seed)
net.add(FCLayer(2, 3))
net.add(ActivationLayer(tanh_act))
net.add(FCLayer(3, 1))
net.add(ActivationLayer(tanh_act))

# setup
net.use(mse, mse_prime)

X = np.array([[[[0, 0]]], [[[0, 1]]], [[[1, 0]]], [[[1, 1]]]])
y = np.array([[[[0]]], [[[1]]], [[[1]]], [[[0]]]])

#train
for i in range(len(X)):
    net.fit(X[i], y[i], epochs=epochs_n, learning_rate=learning_rate)

# test
out = net.predict(X)

### Model for Fed-Learning

In [8]:
# network (for federated learning)
net2 = Network(seed=seed)
net2.add(FCLayer(2, 3))
net2.add(ActivationLayer(tanh_act))
net2.add(FCLayer(3, 1))
net2.add(ActivationLayer(tanh_act))

#setup
net2.use(mse, mse_prime)

#train
s = znp_fed_training(net2, X, y, [str('1')]*len(X))

#test
out2 = s.predict(X)

### Test if the 2 models behave in the same way

In [9]:
np.array_equal(out, out2)

True